In [ ]:
!pip install neo4j-driver

In [ ]:
!pip install wordcloud

In [1]:
import snscrape.modules.twitter as sntwitter
import pandas as pd
import datetime
import re
import time;
from neo4j import GraphDatabase, basic_auth
from wordcloud import WordCloud
import matplotlib.pyplot as plt

## Data acquisition

Vengono recuperati i tweets che contengono la parola gamestop (quindi anche l'hashtag #gamestop) tramite la libreria snscrape.

In [ ]:
#maxTweets = 5000
tweets_list = []

for i,tweet in enumerate(sntwitter.TwitterSearchScraper('gamestop since:2021-01-24 until:2021-01-30').get_items()):
    #if i>maxTweets:
    #    break
    if(i%1000==0):
        print(i)
    tweets_list.append({
        #"date":pd.to_datetime(datetime.datetime.date(tweet.date)),
        "date":tweet.date,
        "tweetId":tweet.id,
        "userId":tweet.user.id,
        "username":tweet.user.username,
        "display_name":tweet.user.displayname,
        "text":tweet.content, 
        "reply_count":tweet.replyCount,
        "like_count":tweet.likeCount,
        "retweet_count":tweet.retweetCount,
        "quote_count":tweet.quoteCount,
        "retweted_tweet":tweet.retweetedTweet,
        "quotedTweet":tweet.quotedTweet
        #"mentionedUsers":tweet.mentionedUsers
    })


In [ ]:
tweet_df = pd.DataFrame(tweets_list)
tweet_df.sort_values(by=['date'], ascending=True, inplace=True)
tweet_df

I dati recuperati vengono subito salvati su file system nel formato originale, quindi non ancora manipolati, per poi averli a disposizione per le successive operazioni

In [ ]:
tweet_df.to_csv('../data/gamestop_tweets_orig.csv', index=False)

Si decide di ripetere la precedente operazione per prelevare ulteriori tweets precedenti a quelli già recuperati

In [ ]:
tweets_list1 = []

for i,tweet in enumerate(sntwitter.TwitterSearchScraper('gamestop since:2021-01-17 until:2021-01-24').get_items()):
    #if i>maxTweets:
    #    break
    if(i%1000==0):
        print(i)
    tweets_list1.append({
        #"date":pd.to_datetime(datetime.datetime.date(tweet.date)),
        "date":tweet.date,
        "tweetId":tweet.id,
        "userId":tweet.user.id,
        "username":tweet.user.username,
        "display_name":tweet.user.displayname,
        "text":tweet.content, 
        "reply_count":tweet.replyCount,
        "like_count":tweet.likeCount,
        "retweet_count":tweet.retweetCount,
        "quote_count":tweet.quoteCount,
        "retweted_tweet":tweet.retweetedTweet,
        "quotedTweet":tweet.quotedTweet,
        "mentionedUsers":tweet.mentionedUsers
    })


Anche in questo caso i risultati vengono subito salvati su file

In [ ]:
tweet_df1 = pd.DataFrame(tweets_list1)
tweet_df1.to_csv('../data/gamestop_tweets_orig_17-24-gen.csv', index=False)

Dopo un'analisi espolorativa del dataset, in particolare sul testo dei tweets, ci si accorge che possono mancare altri tweets di rilevante importanza. Viene ripetuta quindi l'operazione utilizzando le parole chiave GME e Gamestonk

In [ ]:
tweets_list2 = []

for i,tweet in enumerate(sntwitter.TwitterSearchScraper('GME OR Gamestonk since:2021-01-17 until:2021-01-30').get_items()):
    #if i>maxTweets:
    #    break
    if(i%1000==0):
        print(i)
    tweets_list2.append({
        #"date":pd.to_datetime(datetime.datetime.date(tweet.date)),
        "date":tweet.date,
        "tweetId":tweet.id,
        "userId":tweet.user.id,
        "username":tweet.user.username,
        "display_name":tweet.user.displayname,
        "text":tweet.content, 
        "reply_count":tweet.replyCount,
        "like_count":tweet.likeCount,
        "retweet_count":tweet.retweetCount,
        "quote_count":tweet.quoteCount,
        "retweted_tweet":tweet.retweetedTweet,
        "quotedTweet":tweet.quotedTweet
        #"mentionedUsers":tweet.mentionedUsers
    })



In [ ]:
#5 hours long...

In [ ]:
tweet_df2 = pd.DataFrame(tweets_list2)
tweet_df2.to_csv('../data/gamestop_gamestonk_gme_tweets_orig.csv', index=False)

Il motivo del salvataggio dei dati su files nelle operazioni precedenti risiede su fatto di poter avere la possibilità di ripetere le successive operazioni senza dover recuperare di nuovo tutti i dati dalla rete

## Data preparation e data quality

In [3]:
tweet_df = pd.read_csv('../data/gamestop_tweets_orig.csv', dtype=str)
tweet_df1 = pd.read_csv('../data/gamestop_tweets_orig_17-24-gen.csv', dtype=str)
tweet_df2 = pd.read_csv('../data/gamestop_gamestonk_gme_tweets_orig.csv', dtype=str)

<br/><br/>
Unione dei data frames: si tratta semplicemente di fare l'unione dei dati, dal momento che i data frames contengono dati disgiunti, ovvero uno è riferito all'intervallo temporale 17-23 gennaio, uno all'intervallo 24-30 gennaio e l'ultimo comprende il periodo complessivo, ma è stato richiesto con altre keywords. L'unione sicuramente genererà dei duplicati, dato che nel terzo dataframe potrebbero essere presenti tweets già prelevati in precedenza

In [4]:
tweet_df = pd.concat([tweet_df2, tweet_df1, tweet_df], ignore_index=True, sort=True)

In [5]:
tweet_df

,date,display_name,like_count,mentionedUsers,quote_count,quotedTweet,reply_count,retweet_count,retweted_tweet,text,tweetId,userId,username
0,2021-01-17 00:06:58+00:00,CVC Research 🫐,10,NaN,1,NaN,2,0,NaN,$GME https://t.co/B7B3dilRAx,1350595430707466243,3029204524,CVCResearch
1,2021-01-17 00:08:07+00:00,ChuloCharts ☄,0,NaN,0,NaN,0,0,NaN,@MrZackMorris what if?! GME fractal over AMC h...,1350595723402862594,1263589556084813824,ChuloCharts
2,2021-01-17 00:11:27+00:00,CVC Research 🫐,7,NaN,0,NaN,1,0,NaN,$GME https://t.co/N4YBbs3Jdx,1350596559369474049,3029204524,CVCResearch
3,2021-01-17 00:11:36+00:00,CVC Research 🫐,8,NaN,0,NaN,1,0,NaN,$GME https://t.co/mFoHmU8imQ,1350596596942028801,3029204524,CVCResearch
4,2021-01-17 00:13:08+00:00,GameStop millionaire,0,NaN,0,NaN,0,1,NaN,$GME $TSLA i bet you’re his boss now lol https...,1350596982574866434,20292314,KINGDJTRICKS
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1098648,2021-01-29 23:59:46+00:00,Chad Hasty,3,"[User(username='KenPaxtonTX', id=45840174, dis...",0,https://twitter.com/KenPaxtonTX/status/1355248...,1,2,NaN,"At 6:20pm, @KenPaxtonTX will join me on @KFYO,...",1355304663801868290,14927208,ChadHastyRadio
1098649,2021-01-29 23:59:47+00:00,75e Session,246,NaN,0,NaN,0,20,NaN,On va les GameStop,1355304668319141891,264189613,75eSession
1098650,2021-01-29 23:59:49+00:00,Captor Zone,0,NaN,0,NaN,0,0,NaN,https://t.co/CVQRo4Wk30 Candace Owens compares...,1355304676728696834,1552359542,German_Dictator
1098651,2021-01-29 23:59:51+00:00,Casey Jaywork,1,NaN,0,NaN,0,0,NaN,The Gamestop Short Squeeze Explained in 4 Minu...,1355304685100560384,424660548,CaseyJaywork


Dimensione del dataframe contenente tutti i tweets

In [6]:
len(tweet_df.index)

1098653

Verifica della presenza di righe con username o text mancanti

In [7]:
len(tweet_df[tweet_df['username'].isna() | tweet_df['text'].isna()].index)

105

In [8]:
len(tweet_df[tweet_df['username'] == ''].index)

0

In [9]:
len(tweet_df[tweet_df['text'] == ''].index)

0

Username e text sono campi necessari, pertanto vengono rimose le righe laddove tali dati siano mancanti

In [10]:
tweet_df = tweet_df.dropna(subset = ['username'])
tweet_df = tweet_df.dropna(subset = ['text'])

Nuova lunghezza totale del dataset

In [11]:
len(tweet_df.index)

1098548

Verifica della presenza di righe duplicate per tutti i campi

In [13]:
len(tweet_df[tweet_df.duplicated(subset=['tweetId'], keep=False)].index)

111102

Conteggio duplicati mantenendo solo la prima occorrenza

In [171]:
len(tweet_df[tweet_df.duplicated(subset=['tweetId'], keep='first')].index)

71229

Verifica a campione di riga duplicata con lo stesso tweetId

In [172]:
tweet_df[tweet_df['tweetId'] == '1351544479547760642']

,date,display_name,like_count,mentionedUsers,quote_count,quotedTweet,reply_count,retweet_count,retweted_tweet,text,tweetId,userId,username
2481,2021-01-19 14:58:09+00:00,Citron Research,2139,NaN,953,NaN,2010,414,NaN,Tomorrow am at 11:30 EST Citron will livestrea...,1351544479547760642,236953420,CitronResearch
551171,2021-01-19 14:58:09+00:00,Citron Research,2140,NaN,953,NaN,2013,414,NaN,Tomorrow am at 11:30 EST Citron will livestrea...,1351544479547760642,236953420,CitronResearch


<br/><br/>
Rimozione righe duplicate

In [15]:
tweet_df = tweet_df.drop_duplicates(subset=['tweetId'])

In [16]:
len(tweet_df.index)

1027319

Data quality sul campo data: se la conversione genera errore allora la data non è corretta

In [176]:
pd.to_datetime(tweet_df['date'],errors='raise')

0         2021-01-17 00:06:58+00:00
1         2021-01-17 00:08:07+00:00
2         2021-01-17 00:11:27+00:00
3         2021-01-17 00:11:36+00:00
4         2021-01-17 00:13:08+00:00
                     ...           
1098647   2021-01-29 23:59:46+00:00
1098649   2021-01-29 23:59:47+00:00
1098650   2021-01-29 23:59:49+00:00
1098651   2021-01-29 23:59:51+00:00
1098652   2021-01-29 23:59:53+00:00
Name: date, Length: 1027319, dtype: datetime64[ns, UTC]

Verifica presenza di un particolare tweet che pare abbia iniziato la corsa al rialzo, facendo riferimento a wallstreetbets, un subreddit della piattaforma social Reddit'

In [155]:
tweet_df[tweet_df['username'] == 'elonmusk']

,date,display_name,like_count,mentionedUsers,quote_count,quotedTweet,reply_count,retweet_count,retweted_tweet,text,tweetId,userId,username
73468,2021-01-26 21:08:02+00:00,Elon Musk,240616,NaN,9345,NaN,11685,34223,NaN,Gamestonk!! \nhttps://t.co/RZtkDzAewJ,1354174279894642703,44196397,elonmusk


<br/><br/>
Data manipulation testo. Sostituzione caratteri 'a capo' e tabulazione con uno spazio

In [103]:
len(tweet_df[tweet_df['text'].str.contains('\t|\n|\r', regex=True)].index)

211466

In [104]:
tweet_df.replace('\n|\r|\t',' ', regex=True, inplace=True)

In [105]:
len(tweet_df[tweet_df['text'].str.contains('\t|\n|\r', regex=True)].index)

0

<br/><br/>
Conteggio righe rimaste dopo la pulizia

In [107]:
len(tweet_df.index)

1027319

<br/><br/>
Per tutti gli altri campi con valori Na viene inserita una stringa vuota

In [177]:
tweet_df.isna().sum().sum()

2816520

In [178]:
tweet_df.fillna('', inplace=True)

In [179]:
tweet_df

,date,display_name,like_count,mentionedUsers,quote_count,quotedTweet,reply_count,retweet_count,retweted_tweet,text,tweetId,userId,username
0,2021-01-17 00:06:58+00:00,CVC Research 🫐,10,,1,,2,0,,$GME https://t.co/B7B3dilRAx,1350595430707466243,3029204524,CVCResearch
1,2021-01-17 00:08:07+00:00,ChuloCharts ☄,0,,0,,0,0,,@MrZackMorris what if?! GME fractal over AMC h...,1350595723402862594,1263589556084813824,ChuloCharts
2,2021-01-17 00:11:27+00:00,CVC Research 🫐,7,,0,,1,0,,$GME https://t.co/N4YBbs3Jdx,1350596559369474049,3029204524,CVCResearch
3,2021-01-17 00:11:36+00:00,CVC Research 🫐,8,,0,,1,0,,$GME https://t.co/mFoHmU8imQ,1350596596942028801,3029204524,CVCResearch
4,2021-01-17 00:13:08+00:00,GameStop millionaire,0,,0,,0,1,,$GME $TSLA i bet you’re his boss now lol https...,1350596982574866434,20292314,KINGDJTRICKS
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1098647,2021-01-29 23:59:46+00:00,paolo ignazio marong,0,,0,,0,0,,Perdita di 700 punti per la questione #GameSt...,1355304663395004416,1341462944,paoloigna1
1098649,2021-01-29 23:59:47+00:00,75e Session,246,,0,,0,20,,On va les GameStop,1355304668319141891,264189613,75eSession
1098650,2021-01-29 23:59:49+00:00,Captor Zone,0,,0,,0,0,,https://t.co/CVQRo4Wk30 Candace Owens compares...,1355304676728696834,1552359542,German_Dictator
1098651,2021-01-29 23:59:51+00:00,Casey Jaywork,1,,0,,0,0,,The Gamestop Short Squeeze Explained in 4 Minu...,1355304685100560384,424660548,CaseyJaywork


<br/><br/>
Salvataggio delle modifiche su file

In [180]:
tweet_df.to_csv('../data/gamestop_tweets.csv', index=False)

<br/><br/>
Data manipulation: estrazione dei mentioned users dal testo dei tweets
<br/><br/>

In [ ]:
rx = re.compile(r'@([^:^,^@?\s]+)')
mentioned_users = []
for i in range(0, len(tweet_df)):
    try:
        tweetId = tweet_df['tweetId'][i]
        username = tweet_df['username'][i]
        line = tweet_df['text'][i]
        try:
            mentioneds = rx.findall(line)
            for m in mentioneds:
                mentioned_users.append([username, tweetId, m])
        except Exception as e:
            print(e)
    except Exception as e:
        print(e)
    #if(i%100000==0):
    #    print(i)
mentioned_df = pd.DataFrame(mentioned_users, columns = ['tweetUsername', 'tweetId', 'mention'])

<br/><br/>
Verifica e rimozione duplicati nelle mentions

In [112]:
mentioned_df

,tweetUsername,tweetId,mention
0,ChuloCharts,1350595723402862594,MrZackMorris
1,Uncommon_Name1,1350599116519211009,OGxGalv
2,Uncommon_Name1,1350599116519211009,ReeceLongwell
3,Uncommon_Name1,1350599116519211009,chhlss
4,ZevFima,1350601195598065672,wallstreetbets
...,...,...,...
680644,OhNoNotHim5,1355087888174772235,teelokay
680645,oldtreethoughts,1355087897200832512,maneco1964
680646,oldtreethoughts,1355087897200832512,nypost
680647,BradtTunisia,1355087917971075074,reddit


In [113]:
len(mentioned_df[mentioned_df.duplicated(keep=False)].index)

10601

In [114]:
len(mentioned_df[mentioned_df.duplicated(keep='first')].index)

5360

In [115]:
mentioned_df = mentioned_df.drop_duplicates()

<br/><br/>
Salvataggio mentioned users su file

In [116]:
mentioned_df.to_csv('../data/gamestop_mentioned_users_tweets.csv', sep=',', index=False)

In [117]:
mentioned_df

,tweetUsername,tweetId,mention
0,ChuloCharts,1350595723402862594,MrZackMorris
1,Uncommon_Name1,1350599116519211009,OGxGalv
2,Uncommon_Name1,1350599116519211009,ReeceLongwell
3,Uncommon_Name1,1350599116519211009,chhlss
4,ZevFima,1350601195598065672,wallstreetbets
...,...,...,...
680644,OhNoNotHim5,1355087888174772235,teelokay
680645,oldtreethoughts,1355087897200832512,maneco1964
680646,oldtreethoughts,1355087897200832512,nypost
680647,BradtTunisia,1355087917971075074,reddit


<br/><br/>
Verifica del corretto funzionamento della funzione precedente controllando che non ci siano righe con il carattere @ nel campo 'mention

In [118]:
mentioned_df[mentioned_df['mention'].str.contains('@')]

,tweetUsername,tweetId,mention


In [ ]:
#tweet_df[tweet_df['text'].str.contains('#gamestop')]

## Data storage

### Connessione al database Neo4J e inserimento dati

In [137]:
class Neo4jConnection:
    
    def __init__(self, uri, user, pwd):
        self.__uri = uri
        self.__user = user
        self.__pwd = pwd
        self.__driver = None
        try:
            self.__driver = GraphDatabase.driver(self.__uri, auth=(self.__user, self.__pwd))
        except Exception as e:
            print("Failed to create the driver:", e)
        
    def close(self):
        if self.__driver is not None:
            self.__driver.close()
        
    def query(self, query, parameters=None, db=None):
        assert self.__driver is not None, "Driver not initialized!"
        session = None
        response = None
        try: 
            session = self.__driver.session(database=db) if db is not None else self.__driver.session() 
            response = list(session.run(query, parameters))
        except Exception as e:
            print("Query failed:", e)
        finally: 
            if session is not None:
                session.close()
        return response


#conn = Neo4jConnection(uri="bolt://54.90.165.10:7687", 
#                       user="neo4j",              
#                       pwd="clicks-haul-qualifier")

conn = Neo4jConnection(uri="bolt://localhost:7787", 
                       user="neo4j",              
                       pwd="neo4j1")

In [120]:
def insert_data(query, rows, batch_size = 10000):
    # Function to handle the updating the Neo4j database in batch mode.
    
    total = 0
    batch = 0
    start = time.time()
    result = None
    
    while batch * batch_size < len(rows):

        res = conn.query(query, 
                         parameters = {'rows': rows[batch*batch_size:(batch+1)*batch_size].to_dict('records')})
        total += res[0]['total']
        batch += 1
        result = {"total":total, 
                  "batches":batch, 
                  "time":time.time()-start}
        print(result)
        
    return result

In [121]:
def add_tweets(rows, batch_size=10000):
 
   query = '''
   UNWIND $rows as row
   MERGE (u:TwUser {username:row.username})
   MERGE (t:Tweet {tweetId: row.tweetId})
   SET  t.date = row.date,
        t.userId = row.userId,
        t.username = row.username,
        t.display_name = row.display_name,
        t.text = row.text,
        t.reply_count = row.reply_count,
        t.like_count = row.like_count,
        t.retweet_count = row.retweet_count,
        t.quote_count = row.quote_count,
        t.retweeted_tweet = row.retweted_tweet
   MERGE (u)-[:TWEETED]->(t)
 
   
   RETURN count(distinct u) as total
   '''
 
   return insert_data(query, rows, batch_size)

In [122]:
def update_tweet_text(rows, batch_size=10000):
 
   query = '''
   UNWIND $rows as row
   MERGE (t:Tweet {tweetId: row.tweetId})
   SET  t.text = row.text
 
   
   RETURN count(distinct t) as total
   '''
 
   return insert_data(query, rows, batch_size)

In [123]:
def add_mentions(rows, batch_size=10000):
 
   query = '''
    UNWIND $rows as row
    MERGE (u:TwUser {username:row.mention})
    WITH row, u
    MATCH (t:Tweet)
    WHERE t.tweetId = row.tweetId AND u.username = row.mention
    MERGE (t)-[:MENTIONED]->(u)
 
   
    RETURN count(distinct u) as total
    '''
 
   return insert_data(query, rows, batch_size)

In [124]:
def add_user_mentioned_user(rows, batch_size=10000):
 
   query = '''
    UNWIND $rows as row
    MERGE (u:TwUser {username:row.tweetUsername})
    WITH row, u
    MATCH (t:TwUser)
    WHERE t.username = row.mention
    MERGE (u)-[:HAS_MENTIONED]->(t)
 
   
    RETURN count(distinct u) as total
    '''
 
   return insert_data(query, rows, batch_size)

In [125]:
def delete_user_mentioned_user(rows, batch_size=100):
 
   query = '''
    UNWIND $rows as row
    MATCH (n:TwUser {username:row.tweetUsername})-[r:HAS_MENTIONED]->()
    DELETE r
 
   
    RETURN count(distinct r) as total
    '''
 
   return insert_data(query, rows, batch_size)

In [ ]:
conn.query('CREATE CONSTRAINT tweet IF NOT EXISTS ON (a:Tweet) ASSERT a.tweetId IS UNIQUE')

In [ ]:
conn.query('CREATE CONSTRAINT twitterUser IF NOT EXISTS ON (a:TwUser) ASSERT a.username IS UNIQUE')

In [ ]:
add_tweets(tweet_df)

In [ ]:
#update_tweet_text(tweet_df)

In [ ]:
#There is a uniqueness constraint on (:Tweet {tweetId}), so an index is already created that match
#query = '''
#CREATE INDEX FOR (t:Tweet) ON (t.tweetId)
#'''
#conn.query(query)

In [ ]:
add_mentions(mentioned_df)

In [ ]:
add_user_mentioned_user(mentioned_df)

In [ ]:
#delete_user_mentioned_user(mentioned_df)

In [ ]:
conn.query('CREATE INDEX FOR (s:Tweet) ON (s.text)')

### Neo4j graph datascience library

Gli algoritmi di centralità vengono utilizzati per determinare l'importanza di nodi distinti in una rete.<br/>
The Degree Centrality algorithm can be used to find popular nodes within a graph. Degree centrality measures the number of incoming or outgoing (or both) relationships from a node, depending on the orientation of a relationship projection.
<br/>
Viene calcolata la degree centrality per i nodi TwUser

<br/><br/>
Nota: le operazioni seguenti provocano un timeout sulla transazione utilizzando il db remoto, creato sul cloud di Neo4J. Pertanto è stato utilizzato un db locale per i calcoli e successivamente i risultati sono stato inseriti sul db remoto.

In [ ]:
query = '''
CALL gds.degree.stream({
nodeProjection: ['TwUser', 'Tweet'],
relationshipProjection: {
HAS_MENTIONED: {type:'HAS_MENTIONED', orientation:'UNDIRECTED'},
TWEETED: {type:'TWEETED', orientation:'NATURAL'}
}
})
YIELD nodeId, score
RETURN gds.util.asNode(nodeId).username AS unit, score AS degree_centrality
ORDER BY degree_centrality DESC
'''
conn.query(query)

### Data enrichment
Viene inserita in ogni nodo la proprietà 'degree' contenente il valore di degree centrality

In [ ]:
query = '''
CALL gds.degree.write(
{
nodeProjection: ['TwUser'],
relationshipProjection: {
HAS_MENTIONED: {type:'HAS_MENTIONED', orientation:'UNDIRECTED'},
TWEETED: {type:'TWEETED', orientation:'NATURAL'}
},
writeProperty: 'degree'
}
)

'''
conn.query(query)

Creazione indice sul nuovo campo

In [ ]:
conn.query('CREATE INDEX FOR (s:TwUser) ON (s.degree)')

Estrazione valore del campo 'degree' dal db locale per i nodi TwUser e Tweet
<br/>


In [ ]:
query = '''
MATCH (n:TwUser) RETURN n.degree, n.username
'''
result = conn.query(query)
twitterUsers = pd.DataFrame(result, columns={'degree', 'username'})


In [ ]:
twitterUsers.rename(columns={'username':'degree', 'degree':'username'}, inplace=True)
twitterUsers.sort_values(by=['degree'], ascending=False)

In [ ]:
query = '''
MATCH (n:Tweet) RETURN n.degree, n.tweetId
'''
result = conn.query(query)
tweets = pd.DataFrame(result, columns={'degree', 'tweetId'})

In [ ]:
tweets

<br/><br/>
Viene modificato il puntamento al db (da locale a remoto) per poi inserire i valori di 'degree' ai corrispondenti nodi

In [ ]:
def update_twUser(rows, batch_size=5000):
 
    query = '''
    UNWIND $rows as row
    MATCH (p:TwUser {username: row.username})
    SET p.degree = row.degree

    RETURN count(distinct p) as total
    '''
 
    return insert_data(query, rows, batch_size)

In [ ]:
update_twUser(twitterUsers)

In [ ]:
def update_tweet(rows, batch_size=5000):
 
    query = '''
    UNWIND $rows as row
    MATCH (p:Tweet {tweetId: row.tweetId})
    SET p.degree = row.degree

    RETURN count(distinct p) as total
    '''
 
    return insert_data(query, rows, batch_size)

In [ ]:
update_tweet(tweets)

In [ ]:
conn.close()

### Text analysys

Estrazione delle parole più utilizzate nei tweets per poterle poi utilizzare nella selezione dei tweets più rappresentativi per l'intera operazione di incremento del titolo GME

In [130]:
texts = []
def remove_links(row):
    clean_tweet = re.sub("@[A-Za-z0-9_]+","", row['text'])
    clean_tweet = re.sub("#[A-Za-z0-9_]+","", clean_tweet)
    clean_tweet = re.sub(r"http\S+", "", clean_tweet)
    texts.append(clean_tweet)


In [131]:
tweet_df.apply(remove_links, axis=1)

0          None
1          None
2          None
3          None
4          None
           ... 
1098647    None
1098649    None
1098650    None
1098651    None
1098652    None
Length: 1027319, dtype: object

In [132]:
allWords= ' '.join( [twts for twts in texts] )
words = WordCloud().process_text(allWords)

In [133]:
words_df = pd.DataFrame.from_dict(words, orient='index')
#words

In [134]:
words_df.sort_values(by=[0], ascending=False, inplace=True)

In [135]:
pd.set_option('display.max_rows', 200)

In [136]:
display(words_df.head(100))

,0
GameStop,70247
hedge fund,55490
GME AMC,48777
GameStop stock,48345
GME,46470
Wall Street,38586
s,33387
going,32031
AMC GME,30192
now,29546
